In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
import gc

# ===============================================================
# 1. ⚙️ 配置 
# ===============================================================
class CFG:
    # --- 路径  ---
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TEST_CSV = os.path.join(BASE_PATH, 'test.csv')
    TEST_IMAGE_DIR = os.path.join(BASE_PATH, 'test')
    
    # 包含 5 个 .pth 文件的目录
    MODEL_DIR = '/kaggle/input/csiro/'
    SUBMISSION_FILE = 'submission.csv'
    
    # --- 模型设置 ---
    MODEL_NAME = 'convnext_tiny'
    IMG_SIZE = 768              
    
    # --- 推理设置 ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 1 # 推理时可以增大学习率
    NUM_WORKERS = 1
    N_FOLDS = 5
    
    # --- 目标与损失 (必须匹配) ---
    # 模型预测的 3 个目标
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    
    # 提交所需的 5 个目标
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

print(f"使用设备: {CFG.DEVICE}")
print(f"模型骨干: {CFG.MODEL_NAME}")
print(f"推理图片尺寸: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")


# ===============================================================
# 2. 🏞️ 数据增强
# ===============================================================
from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip
)

def get_tta_transforms():
    """
    返回一个用于 TTA 的 transform pipeline 列表。
    每个 pipeline 都是图像的一个不同“视角”。
    """
    
    # 这是基本的标准化步骤
    base_transforms = [
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ]
    
    # -----------------
    # 视角 1: 原始图像 
    # -----------------
    original_view = Compose([
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # 视角 2: 水平翻转 
    # -----------------
    hflip_view = Compose([
        HorizontalFlip(p=1.0), # 始终翻转
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    # -----------------
    # 视角 3: 垂直翻转 
    # -----------------
    vflip_view = Compose([
        VerticalFlip(p=1.0), # 始终翻转
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        *base_transforms
    ])
    
    return [original_view, hflip_view, vflip_view]

print("已定义 get_tta_transforms() 函数。")


class TestBiomassDataset(Dataset):
    """
    用于测试图像的自定义 Dataset (双流策略)。
    修改为接受一个特定的 TTA transform pipeline。
    """
    def __init__(self, df, transform_pipeline, image_dir):
        self.df = df
        self.transforms = transform_pipeline 
        self.image_dir = image_dir
        self.image_paths = df['image_path'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. 获取信息
        img_path_suffix = self.image_paths[idx]
        
        # 2. 读取原始图像 (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        
        image = cv2.imread(full_path)
        if image is None:
            print(f"警告: 无法读取图像: {full_path}。返回黑色图像。")
            image = np.zeros((1000, 2000, 3), dtype=np.uint8)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. 裁剪成 2 张图像 (左侧和右侧)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. 应用 TTA Transform (对两者使用相同的 Transform)
        img_left_tensor = self.transforms(image=img_left)['image']
        img_right_tensor = self.transforms(image=img_right)['image']
        
        # 5. 返回
        return img_left_tensor, img_right_tensor

# ===============================================================
# 4. 🧠 模型架构
# ===============================================================
class BiomassModel(nn.Module):
    """
    模型架构 (双流, 三输出)
    必须与训练文件完全一致。
    """
    def __init__(self, model_name, pretrained, n_targets=3):
        super(BiomassModel, self).__init__()
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg'
        )
        
        self.n_features = self.backbone.num_features
        self.n_combined_features = self.n_features * 2
        
        # --- Dry_Total_g 的输出头 ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- GDM_g 的输出头 ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Dry_Green_g 的输出头 ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, img_left, img_right):
        features_left = self.backbone(img_left)
        features_right = self.backbone(img_right)
        combined = torch.cat([features_left, features_right], dim=1)
        
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        return out_total, out_gdm, out_green


def predict_one_view(models_list, test_loader, device):
    """
    子函数: 为单个 TTA 视角运行 5 折集成预测。
    """
    view_preds_3 = {'total': [], 'gdm': [], 'green': []}
    
    with torch.no_grad():
        for (img_left, img_right) in tqdm(test_loader, desc="  Predicting View", leave=False):
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            
            batch_preds_3_folds = {'total': [], 'gdm': [], 'green': []}
            
            # 1. 5 折集成循环
            for model in models_list:
                pred_total, pred_gdm, pred_green = model(img_left, img_right)
                batch_preds_3_folds['total'].append(pred_total.cpu())
                batch_preds_3_folds['gdm'].append(pred_gdm.cpu())
                batch_preds_3_folds['green'].append(pred_green.cpu())
            
            # 2. 计算 5 折的平均值
            avg_pred_total = torch.mean(torch.stack(batch_preds_3_folds['total']), dim=0)
            avg_pred_gdm = torch.mean(torch.stack(batch_preds_3_folds['gdm']), dim=0)
            avg_pred_green = torch.mean(torch.stack(batch_preds_3_folds['green']), dim=0)
            
            view_preds_3['total'].append(avg_pred_total.numpy())
            view_preds_3['gdm'].append(avg_pred_gdm.numpy())
            view_preds_3['green'].append(avg_pred_green.numpy())

    # 3. 合并此视角的批处理结果
    preds_np = {
        'total': np.concatenate(view_preds_3['total']).flatten(),
        'gdm':   np.concatenate(view_preds_3['gdm']).flatten(),
        'green': np.concatenate(view_preds_3['green']).flatten()
    }
    return preds_np


def run_inference_with_tta():
    """
    主推理函数, 执行 TTA x 集成。
    """
    print(f"\n{'='*50}")
    print(f"🚀 开始推理 (使用 TTA) 🚀")
    print(f"{'='*50}")

    # --- 1. 加载测试数据 ---
    print(f"正在加载 {CFG.TEST_CSV}...")
    try:
        test_df_long = pd.read_csv(CFG.TEST_CSV)
        test_df_unique = test_df_long.drop_duplicates(subset=['image_path']).reset_index(drop=True)
        print(f"找到 {len(test_df_unique)} 张唯一的测试图像。")
    except FileNotFoundError:
        print(f"错误: 未找到 {CFG.TEST_CSV}")
        return None, None, None

    # --- 2. 加载 5 个模型 (集成) ---
    print("\n正在加载 5 个已训练的模型...")
    models_list = []
    for fold in range(CFG.N_FOLDS):
        model_path = os.path.join(CFG.MODEL_DIR, f'best_model_fold{fold}.pth')
        if not os.path.exists(model_path):
            print(f"错误: 未找到模型文件: {model_path}")
            return None, None, None
        model = BiomassModel(CFG.MODEL_NAME, pretrained=False)
        try:
            model.load_state_dict(torch.load(model_path, map_location=CFG.DEVICE))
        except RuntimeError:
            state_dict = torch.load(model_path, map_location=CFG.DEVICE)
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k.replace('module.', '')
                new_state_dict[name] = v
            model.load_state_dict(new_state_dict)
        model.eval()
        model.to(CFG.DEVICE)
        models_list.append(model)
    print(f"✓ 成功加载 {len(models_list)} 个模型。")

    # --- 3. TTA 循环  ---
    tta_transforms = get_tta_transforms()
    print(f"\n开始使用 {len(tta_transforms)} 个 TTA 视角进行预测...")
    
    all_tta_view_preds = [] 

    for i, tta_transform in enumerate(tta_transforms):
        print(f"--- 正在运行 TTA 视角 {i+1}/{len(tta_transforms)} ---")
        
        # 为此 TTA 视角创建新的 Dataset/Loader
        test_dataset = TestBiomassDataset(
            df=test_df_unique,
            transform_pipeline=tta_transform,
            image_dir=CFG.TEST_IMAGE_DIR
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=CFG.BATCH_SIZE,
            shuffle=False,
            num_workers=CFG.NUM_WORKERS,
            pin_memory=True
        )
        
        # 为此视角运行 5 折集成
        view_preds_np = predict_one_view(models_list, test_loader, CFG.DEVICE)
        all_tta_view_preds.append(view_preds_np)
        print(f"✓ 完成 TTA 视角 {i+1}")

    # --- 4. 集成 (平均) TTA 结果 ---
    print("\n正在集成 TTA 视角的结果...")
    final_ensembled_preds = {
        'total': np.mean([d['total'] for d in all_tta_view_preds], axis=0),
        'gdm':   np.mean([d['gdm'] for d in all_tta_view_preds], axis=0),
        'green': np.mean([d['green'] for d in all_tta_view_preds], axis=0)
    }
    
    print("✓ 预测完成。")
    
    del models_list, test_loader, test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    
    return final_ensembled_preds, test_df_long, test_df_unique
# ===============================================================
# 6. ✍️ 创建提交文件的函数
# ===============================================================
def create_submission(preds_np, test_df_long, test_df_unique):
    """
    此函数接收 3 个集成后的预测结果，
    计算剩余 2 个预测结果，
    并格式化提交文件。
    """
    if preds_np is None:
        print("因上述错误，跳过创建提交文件。")
        return

    print("\n正在后处理并创建提交文件...")

    # 1. 获取 3 个集成后的预测结果
    pred_total_final = preds_np['total']
    pred_gdm_final = preds_np['gdm']
    pred_green_final = preds_np['green']

    # 2. 计算剩余 2 个目标 (后处理)
    # 使用 np.maximum(0, ...) 确保没有负值
    pred_clover_final = np.maximum(0, pred_gdm_final - pred_green_final)
    pred_dead_final = np.maximum(0, pred_total_final - pred_gdm_final)

    # 3. 创建一个包含 5 个预测结果的“宽”DataFrame
    # (确保 5 个列的顺序与 CFG.ALL_TARGET_COLS 相同)
    preds_wide_df = pd.DataFrame({
        'image_path': test_df_unique['image_path'],
        'Dry_Green_g': pred_green_final,
        'Dry_Dead_g': pred_dead_final,
        'Dry_Clover_g': pred_clover_final,
        'GDM_g': pred_gdm_final,
        'Dry_Total_g': pred_total_final
    })

    # 4. “逆透视” DataFrame (转换为“长”格式)
    # 将其从 5 列转换为“长”格式 (与 sample_submission 类似)
    preds_long_df = preds_wide_df.melt(
        id_vars=['image_path'],
        value_vars=CFG.ALL_TARGET_COLS, # 5 个目标列
        var_name='target_name',        # 目标名称列
        value_name='target'            # 预测值列
    )

    # 5. 与原始 test.csv 文件 (test_df_long) 合并
    # 这是获取正确 'sample_id' 的关键步骤
    # (例如: 'ID1001187975__Dry_Clover_g')
    submission_df = pd.merge(
        test_df_long[['sample_id', 'image_path', 'target_name']],
        preds_long_df,
        on=['image_path', 'target_name'],
        how='left'
    )

    # 6. 清理并保存
    # 仅保留所需的 2 个列
    submission_df = submission_df[['sample_id', 'target']]
    
    # 保存文件
    submission_df.to_csv(CFG.SUBMISSION_FILE, index=False)

    print(f"\n🎉 完成! 提交文件已保存至: {CFG.SUBMISSION_FILE}")
    print("--- 提交文件的前 5 行 ---")
    print(submission_df.head())
    print("\n--- 提交文件的后 5 行 ---")
    print(submission_df.tail())
    
# ===============================================================
# 8. 🏁 运行程序 (已修改)
# ===============================================================
if __name__ == "__main__":
    # 1. 运行预测 (已包含 TTA)
    all_preds_np, df_long, df_unique = run_inference_with_tta()
    
    # 2. 创建提交文件 (create_submission 函数保持不变)
    create_submission(all_preds_np, df_long, df_unique)